In [3]:
from torch.nn import CrossEntropyLoss
from transformer import TransformerMT
from torch import optim
from data_loader import *
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = EnFrDataset(used_abridged_data=True, max_seq_length=100)

train_dataloader = DataLoader(data, batch_size=32, shuffle=False, num_workers=0)

transformer_mt = TransformerMT(
    source_vocabulary_size=data.get_src_lang_size(),
    target_vocabulary_size=data.get_tgt_lang_size(),
    embedding_size=512,
    max_num_embeddings=100,
    num_attention_heads=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    linear_layer_size=2048,
    dropout=0.1,
    activation='relu',
    layer_norm_eps=1e-5,
    batch_first=False,
    norm_first=False,
    bias=True
)
transformer_mt.to(device)

optimizer = optim.Adam(transformer_mt.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

loss_criterion = CrossEntropyLoss(ignore_index=2)

for e in range(20):
    transformer_mt
    running_loss = []
    for en_token_ids, fr_token_ids in train_dataloader:
        en_token_ids.to(device)
        print(en_token_ids.shape)
        fr_token_ids.to(device)
        print(fr_token_ids.shape)
    
        optimizer.zero_grad()
    
        # forward + backward + optimize
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = transformer_mt.generate_mask(en_token_ids, fr_token_ids)
        
        output = transformer_mt(src=en_token_ids, tgt=fr_token_ids, src_mask=src_mask, tgt_mask=tgt_mask, src_key_padding_mask=src_padding_mask, tgt_key_padding_mask=tgt_padding_mask)
        
        loss = loss_criterion(output.contiguous().view(-1, data.get_tgt_lang_size()), fr_token_ids[:, 1:].contiguous().view(-1))
    
        loss.backward()
        optimizer.step()
    
        running_loss += [loss.item()]
    
    print(f"epoch: {e+1}, epoch loss: {running_loss}")

Reading the dataframe and storing untokenized pairs...


100%|██████████| 5000/5000 [00:00<00:00, 167660.83it/s]


Adding sentences to Langs amd geting data pairs...


100%|██████████| 5000/5000 [00:01<00:00, 4796.57it/s]


Creating tokenized pairs of english and french sentences...


100%|██████████| 5000/5000 [00:00<00:00, 10239.26it/s]


torch.Size([32, 100])
torch.Size([32, 100])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([100, 32])
torch.Size([100, 32])


RuntimeError: The shape of the 2D attn_mask is torch.Size([32, 32]), but should be (100, 100).